# 🌐 Macro‑Sentinel · Colab Notebook
*Alpha‑Factory v1 👁️✨ — Cross‑asset macro risk radar*

### Why this notebook?

Run the full **Macro‑Sentinel** agent stack in <10 min without Docker.
Ideal for quick experimentation, hackathons, classrooms, or due‑diligence.

| Mode | LLM | Data feeds |
|------|-----|------------|
| **Offline** (default) | Mixtral‑8x7B (Ollama) | bundled CSV snapshots |
| **Online**            | GPT‑4o (OpenAI)       | FRED API, Fed RSS, on‑chain flows |

---

## 0 · Runtime check

In [ ]:
!nvidia-smi -L || echo '🔹 GPU not detected — running on CPU'

## 1 · Clone repo & install Python deps
*(≈ 90 s; wheels cached by Colab)*

In [ ]:
%%bash
set -e
if [ ! -d AGI-Alpha-Agent-v0 ]; then
  git clone --depth 1 https://github.com/MontrealAI/AGI-Alpha-Agent-v0.git
fi
pip -q install -U openai_agents==0.2.4 gradio aiohttp psycopg2-binary                    qdrant-client rich pretty_errors                    ollama-py~=0.1.4


### 🛜 Optional: pull Mixtral for offline mode (~4 GB)

In [ ]:
%%bash
python - <<'PY'
import os, subprocess, json, shutil, pathlib, sys
if not os.getenv("OPENAI_API_KEY"):
    try:
        subprocess.run(["ollama", "serve"], check=False, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        subprocess.run(["ollama", "pull", "mixtral:instruct"], check=True)
    except FileNotFoundError:
        print("⚠️ Ollama not found; offline LLM will not work.")
PY


## 2 · Configure credentials & runtime flags

In [ ]:
import os, getpass, json
def _set(k,v):
    if v is not None:
        os.environ[k]=v

_set('OPENAI_API_KEY', getpass.getpass('🔑 OpenAI key (blank for offline): '))
_set('FRED_API_KEY',  '')
_set('LIVE_FEED',     input('Real‑time feeds? (0/1) → ') or '0')
os.environ['DEFAULT_PORTFOLIO_USD'] = '2000000'

print(json.dumps({k:os.getenv(k,'') for k in ['OPENAI_API_KEY','FRED_API_KEY','LIVE_FEED']}, indent=2))


## 3 · Launch Macro‑Sentinel dashboard

In [ ]:
import subprocess, re, pathlib, queue, threading, sys, time, textwrap, os
root = pathlib.Path('AGI-Alpha-Agent-v0/alpha_factory_v1/demos/macro_sentinel')
proc = subprocess.Popen([sys.executable, 'agent_macro_entrypoint.py'],
                        cwd=root,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.STDOUT,
                        text=True, bufsize=1)

link_q = queue.Queue()
def _tail():
    for line in proc.stdout:
        print(line, end='')
        m = re.search(r'(https://[\w.-]+\.gradio\.live)', line)
        if m: link_q.put(m.group(1))
threading.Thread(target=_tail, daemon=True).start()

print('⏳ Waiting for Gradio tunnel...')
url = link_q.get()
print(f'🎯 Open dashboard → {url}')


## 4 · Programmatic agent call (optional)

In [ ]:
import asyncio, importlib, sys, json, pandas as pd
sys.path.append('AGI-Alpha-Agent-v0/alpha_factory_v1/demos/macro_sentinel')
import agent_macro_entrypoint as ms

async def cycle():
    evt  = await ms.macro_event()
    risk = await ms.mc_risk(evt)
    print('VaR 5 %:', risk['hedge']['metrics']['var'])
    df = pd.DataFrame(risk['scenarios'])
    return df
await cycle()


## 5 · Chat over A2A protocol (bonus)

In [ ]:
from google.a2a import AgentClient
client = AgentClient(endpoint='http://localhost:7864/a2a')
resp = client.chat({'query':'Summarize latest Fed speech & hedge suggestion'})
print(resp['text'])


## 6 · Graceful shutdown

In [ ]:
proc.terminate(); print('✅ Sentinel stopped')

---
© 2025 **MONTREAL.AI** • MIT License